## **Federated Learning for attack classification: 3 nodes sharing gradients**

IDs from this file = **id0xy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Preprocessing for test and training datasets (to be able to train and make predictions to evaluate)
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Creating categories dataframe
    data_labels = pd.DataFrame()

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)
        
    # Transform to One Hot Encoding the Categories - normal, dos, reconnaissance, generic, exploits, worms, fuzzers, analysis, backdoor, shellcode
    data_labels.insert(0, 'normal', data['attack_cat'].replace('normal', 1).replace(['dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(1, 'dos', data['attack_cat'].replace('dos', 1).replace(['normal', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(2, 'reconnaissance', data['attack_cat'].replace('reconnaissance', 1).replace(['normal', 'dos', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(3, 'generic', data['attack_cat'].replace('generic', 1).replace(['normal', 'dos', 'reconnaissance', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(4, 'exploits', data['attack_cat'].replace('exploits', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(5, 'worms', data['attack_cat'].replace('worms', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(6, 'fuzzers', data['attack_cat'].replace('fuzzers', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'analysis', 'backdoor', 'shellcode'], 0))
    data_labels.insert(7, 'analysis', data['attack_cat'].replace('analysis', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'backdoor', 'shellcode'], 0))
    data_labels.insert(8, 'backdoor', data['attack_cat'].replace('backdoor', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'shellcode'], 0))
    data_labels.insert(9, 'shellcode', data['attack_cat'].replace('shellcode', 1).replace(['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor'], 0))

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Generate 2 new columns to fit with training
    auxCol=data_features['sbytes']
    auxCol=0

      # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, labels = [i for i in range(10)], target_names=['normal', 'dos', 'reconnaissance', 'generic', 'exploits', 'worms', 'fuzzers', 'analysis', 'backdoor', 'shellcode'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### Experiments with datasets

#### Dataset 3A

In [5]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part3.csv')
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3268/797642493.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3268/797642493.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3268/797642493.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3268/797642493.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=

In [6]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [7]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id000.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [8]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [9]:
global_model = build_model((24,1,1))

In [10]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [11]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [12]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id000.hdf5')

Epoch 1/15


54/54 [==============================] - 23s 418ms/step - loss: 0.7425 - accuracy: 0.9000 - val_loss: 7.6566 - val_accuracy: 0.3314
Epoch 2/15
54/54 [==============================] - 29s 544ms/step - loss: 0.0514 - accuracy: 0.9920 - val_loss: 8.1478 - val_accuracy: 0.3315
Epoch 3/15
54/54 [==============================] - 27s 509ms/step - loss: 0.0302 - accuracy: 0.9926 - val_loss: 8.4146 - val_accuracy: 0.3317
Epoch 4/15
54/54 [==============================] - 27s 505ms/step - loss: 0.0274 - accuracy: 0.9925 - val_loss: 8.5737 - val_accuracy: 0.3317
Epoch 5/15
54/54 [==============================] - 33s 612ms/step - loss: 0.0259 - accuracy: 0.9926 - val_loss: 8.6774 - val_accuracy: 0.3317
Epoch 6/15
54/54 [==============================] - 32s 602ms/step - loss: 0.0247 - accuracy: 0.9927 - val_loss: 8.7664 - val_accuracy: 0.3317
Epoch 7/15
54/54 [==============================] - 39s 716ms/step - loss: 0.0234 - accuracy: 0.9927 - val_loss: 8.8369 - val_accuracy: 0.3324
Epoch 8/15

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


4279/4279 [==============================] - 89s 21ms/step - loss: 1.7159 - accuracy: 0.90100s - loss: 1.7166 - accuracy: 0.


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

...

c:\Users\UX430\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


4279/4279 [==============================] - 53s 12ms/step - loss: 3.0947 - accuracy: 0.5345


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

1721/1721 [==============================] - 17s 10ms/step - loss: 16.6600 - accuracy: 0.5501 1s 


c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\UX430\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [13]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[1.7159273624420166, 7.04463529586792], [2.1918957233428955, 9.16387939453125], [2.3710083961486816, 8.643190383911133], [2.8661272525787354, 9.455855369567871], [3.008028268814087, 10.14539623260498], [2.9826226234436035, 11.758312225341797], [2.965424060821533, 13.161545753479004], [2.6688168048858643, 14.4466552734375], [3.215939998626709, 15.619219779968262], [3.094697952270508, 16.659975051879883]]
Accuracy for iterations:  [[0.9010415077209473, 0.5553658604621887], [0.9164524078369141, 0.558344841003418], [0.6898463368415833, 0.5581995248794556], [0.5299088358879089, 0.5576727390289307], [0.5295217633247375, 0.5561469197273254], [0.5416678786277771, 0.5544031262397766], [0.5781574249267578, 0.5529862642288208], [0.6938195824623108, 0.5523686408996582], [0.5346197485923767, 0.5508065223693848], [0.534459114074707, 0.5500617623329163]]
F1 for iterations:  [[0.8644252689061761, 0.45439290560040924], [0.879090638864813, 0.42621753868101747], [0.6356918065866827

#### Dataset 3B 

In [14]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3268/1167559547.py:2: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3268/1167559547.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3B-Part2.csv')


In [15]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [16]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id001.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [17]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [18]:
global_model = build_model((24,1,1))

In [19]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [20]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [21]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id001.hdf5')

Epoch 1/15
12/57 [=====>........................] - ETA: 13s - loss: 1.9652 - accuracy: 0.9167

KeyboardInterrupt: 

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### Dataset 3C

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id002.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id002.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### Dataset 3D

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3D-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3D-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3D-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id003.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id003.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### DATASET 3E

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3E-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id004.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id004.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### DATASET 3F

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3F-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id005.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id005.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### DATASET 3G

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3G-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 15

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id006.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id006.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

### Experimenting with epochs and iterations 

Use of dataset 3C, considering it contains an equal partition of the data between the 3 nodes. 

#### #Epochs = 10, #Iterations = 10

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 10

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 10

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id010.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])



global_model.save('C:/Users/UX430/Documents/thesis/code/models/id010.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

#### #Epochs = 5, #Iterations = 5

In [ ]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-3C-Part3.csv')

In [ ]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 5

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'id011.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
def aggregate(grad_list):
    avg_grad = np.mean(grad_list, axis=0)
    return avg_grad

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.keras.losses.categorical_crossentropy(y, predictions)
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/id011.hdf5')

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)